In [6]:
###Correlation between stations.

from viresclient import SwarmRequest
import hapiclient
from hapiclient.util import pythonshell
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from chaosmagpy import load_CHAOS_matfile
from chaosmagpy.model_utils import synth_values
from chaosmagpy.data_utils import mjd2000 # Modified Julian date
import tqdm
from sklearn.cluster import KMeans

import utils

In [7]:
###Step 1: get current directory and set PATH variable to where data will be saved.

import os

# Get the current working directory
current_path = os.getcwd()

# Print the current working directory
print("Current working directory:", current_path)

#define path to data folder where data will be placed. Print current path if you don't know what it is!
PATH = '/mnt/c/users/s1725621/Documents/PhDCode/Git/dBdtCorrelation/data/'

Current working directory: /mnt/c/users/s1725621/Documents/PhDCode/Git/dBdtCorrelation/dBdtCorrelation


In [8]:
###Step 2: Import dataset from csv
observatory='NGK'
obs=pd.read_csv(PATH + observatory +'19982020.csv',index_col='Timestamp')###Observatory Dataset downloaded
obs=obs.to_xarray() #Into xarray

print(obs)

<xarray.Dataset>
Dimensions:    (Timestamp: 11568960)
Coordinates:
  * Timestamp  (Timestamp) object '1998-01-01 00:00:00' ... '2019-12-31 23:59...
Data variables:
    B_N        (Timestamp) float64 -3.773 -4.174 -4.374 ... 15.56 15.66 15.86
    B_E        (Timestamp) float64 27.4 27.39 27.39 ... -24.61 -24.71 -24.92
    B_C        (Timestamp) float64 19.33 19.43 19.53 ... -41.75 -41.85 -41.85
    B_H        (Timestamp) float64 -2.636 -3.036 -3.237 ... 13.59 13.68 13.87
    dBdt       (Timestamp) float64 nan -0.4004 -0.2003 ... 0.09301 0.1854


In [9]:
###Step 2: Import dataset from csv
observatory='LER'
obs=pd.read_csv(PATH + observatory +'19982020.csv',index_col='Timestamp')###Observatory Dataset downloaded
obs=obs.to_xarray() #Into xarray

print(obs)

<xarray.Dataset>
Dimensions:    (Timestamp: 11568960)
Coordinates:
  * Timestamp  (Timestamp) object '1998-01-01 00:00:00' ... '2019-12-31 23:59...
Data variables:
    B_N        (Timestamp) float64 2.986 2.686 2.486 2.725 ... 5.127 5.127 5.328
    B_E        (Timestamp) float64 -30.74 -30.77 -30.79 ... 22.53 22.63 22.83
    B_C        (Timestamp) float64 12.41 12.71 12.71 13.01 ... -27.3 -27.3 -27.4
    B_H        (Timestamp) float64 -4.953 -5.255 -5.456 ... 11.89 11.89 12.09
    dBdt       (Timestamp) float64 nan -0.3024 -0.201 ... 0.001359 0.203


# Looping over all observatories and producing .csv file of dBdt.

In [10]:
###Step 3: loop over all data and download dbdt for each one.
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(PATH) if isfile(join(PATH, f))]

In [11]:
allobsdbdt = pd.DataFrame()

allobsdbdt.index = obs['Timestamp']

In [12]:
print(onlyfiles)
for i in tqdm.tqdm(onlyfiles):
    obsiname=i[0:3]
    obsi=pd.read_csv(PATH + i,index_col='Timestamp')###Observatory Dataset downloaded
    #allobsdbdt = allobsdbdt.assign(obsiname=obsi['dBdt'])
    allobsdbdt[obsiname] = obsi['dBdt']
    print(obsi)

['ABG19982020.csv', 'ABK19982020.csv', 'BDV19982020.csv', 'BEL19982020.csv', 'BLC19982020.csv', 'BMT19982020.csv', 'BOU19982020.csv', 'BRW19982020.csv', 'BSL19982020.csv', 'CLF19982020.csv', 'CMO19982020.csv', 'CNB19982020.csv', 'ESK19982020.csv', 'EYR19982020.csv', 'FCC19982020.csv', 'FRD19982020.csv', 'FRN19982020.csv', 'FUR19982020.csv', 'GDH19982020.csv', 'GUA19982020.csv', 'GUI19982020.csv', 'HAD19982020.csv', 'HER19982020.csv', 'HLP19982020.csv', 'HON19982020.csv', 'HRB19982020.csv', 'IQA19982020.csv', 'IRT19982020.csv', 'KAK19982020.csv', 'LER19982020.csv', 'MEA19982020.csv', 'MMB19982020.csv', 'NAQ19982020.csv', 'NCK19982020.csv', 'NEW19982020.csv', 'NGK19982020.csv', 'NUR19982020.csv', 'OTT19982020.csv', 'PHU19982020.csv', 'RES19982020.csv', 'SBA19982020.csv', 'SIT19982020.csv', 'SJG19982020.csv', 'SOD19982020.csv', 'SPT19982020.csv', 'STJ19982020.csv', 'TAM19982020.csv', 'THL19982020.csv', 'THY19982020.csv', 'TUC19982020.csv', 'VIC19982020.csv', 'WNG19982020.csv', 'YKC1998202

  2%|███▎                                                                                                                                                                         | 1/53 [00:48<42:20, 48.85s/it]

                           B_N         B_E        B_C       B_H      dBdt
Timestamp                                                                
1998-01-01 00:00:00        NaN         NaN        NaN       NaN       NaN
1998-01-01 00:01:00        NaN         NaN        NaN       NaN       NaN
1998-01-01 00:02:00        NaN         NaN        NaN       NaN       NaN
1998-01-01 00:03:00        NaN         NaN        NaN       NaN       NaN
1998-01-01 00:04:00        NaN         NaN        NaN       NaN       NaN
...                        ...         ...        ...       ...       ...
2019-12-31 23:55:00 -12.286700  445.590274 -43.512401 -8.662295 -0.001343
2019-12-31 23:56:00 -12.286742  445.690325 -43.512609 -8.661586  0.000709
2019-12-31 23:57:00 -12.186582  445.490377 -43.612615 -8.562931  0.098654
2019-12-31 23:58:00 -12.086625  445.690428 -43.612621 -8.461474  0.101458
2019-12-31 23:59:00 -12.086262  445.390480 -43.812829 -8.463365 -0.001891

[11328480 rows x 5 columns]


  4%|██████▌                                                                                                                                                                      | 2/53 [01:43<44:38, 52.52s/it]

                           B_N        B_E        B_C        B_H      dBdt
Timestamp                                                                
1998-01-01 00:00:00  -3.994639  19.770414   8.523948  -3.999403       NaN
1998-01-01 00:01:00  -3.795777  21.070346   9.024340  -3.691076  0.308327
1998-01-01 00:02:00  -4.496682  21.570279   9.422655  -4.347040 -0.655964
1998-01-01 00:03:00  -3.799206  21.870211  10.524200  -3.626624  0.720416
1998-01-01 00:04:00  -3.098268  22.170144  10.125749  -2.902753  0.723871
...                        ...        ...        ...        ...       ...
2019-12-31 23:55:00  29.536325 -17.805848 -35.932921  28.745247  0.410346
2019-12-31 23:56:00  29.735901 -17.805937 -35.732559  28.942085  0.196838
2019-12-31 23:57:00  29.735708 -17.906026 -35.632660  28.925416 -0.016670
2019-12-31 23:58:00  29.935746 -18.306116 -35.632298  29.056857  0.131442
2019-12-31 23:59:00  30.136015 -18.706205 -35.731936  29.188543  0.131685

[11567520 rows x 5 columns]


...

                           B_N        B_E        B_C        B_H      dBdt
Timestamp                                                                
1998-01-01 00:00:00  -6.382872 -29.872785 -44.846848 -14.867246       NaN
1998-01-01 00:01:00  -8.380568 -29.772684 -45.553370 -16.712771 -1.845525
1998-01-01 00:02:00  -8.580275 -29.872583 -45.653893 -16.934517 -0.221746
1998-01-01 00:03:00  -8.380651 -29.572482 -45.553083 -16.644989  0.289528
1998-01-01 00:04:00  -8.680357 -29.572381 -45.653939 -16.926919 -0.281929
...                        ...        ...        ...        ...       ...
2019-12-31 23:55:00  26.617807   3.991820  51.507792  25.922946  0.000340
2019-12-31 23:56:00  26.518478   4.091884  51.307667  25.855331 -0.067615
2019-12-31 23:57:00  26.418814   4.191947  51.207541  25.787396 -0.067935
2019-12-31 23:58:00  26.219151   4.292010  51.107082  25.623401 -0.163995
2019-12-31 23:59:00  25.819823   4.592073  50.905957  25.323183 -0.300218

[11494080 rows x 5 columns]


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊   | 52/53 [34:52<00:39, 39.67s/it]

                           B_N        B_E        B_C        B_H      dBdt
Timestamp                                                                
1998-01-01 00:00:00  -2.660706 -83.314043  16.060248  -3.227075       NaN
1998-01-01 00:01:00  -3.663914 -83.313965  17.056970  -4.230283 -1.003207
1998-01-01 00:02:00  -3.660714 -84.313886  16.056910  -4.227948  0.002335
1998-01-01 00:03:00  -3.660721 -83.313807  16.056844  -4.227089  0.000859
1998-01-01 00:04:00  -4.663929 -84.313728  17.053567  -5.231162 -1.004073
...                        ...        ...        ...        ...       ...
2019-12-31 23:55:00  14.700897 -18.489037 -38.869090  14.529952  0.111224
2019-12-31 23:56:00  14.700905 -18.489142 -38.869190  14.529954  0.000002
2019-12-31 23:57:00  14.800912 -18.489246 -38.868969  14.629803  0.099849
2019-12-31 23:58:00  14.900920 -18.589350 -38.868749  14.724125  0.094322
2019-12-31 23:59:00  15.000927 -18.789454 -38.868528  14.812921  0.088796

[11566080 rows x 5 columns]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53/53 [35:21<00:00, 40.03s/it]

                           B_N        B_E        B_C        B_H      dBdt
Timestamp                                                                
1998-01-01 00:00:00  29.058687 -10.862854 -46.077345  20.665963       NaN
1998-01-01 00:01:00  31.559113  -7.662753 -46.270347  24.293328  3.627365
1998-01-01 00:02:00  34.055956  -8.162652 -45.163354  26.337694  2.044366
1998-01-01 00:03:00  33.753636  -8.462551 -44.364078  25.936282 -0.401412
1998-01-01 00:04:00  32.452973  -8.862450 -44.167555  24.589374 -1.346908
...                        ...        ...        ...        ...       ...
2019-12-31 23:55:00  -0.067503 -15.943511  20.760312  -3.200538  0.152552
2019-12-31 23:56:00   0.131567 -15.443500  21.061034  -2.867739  0.332799
2019-12-31 23:57:00  -0.069638 -15.043489  21.460653  -2.947115 -0.079376
2019-12-31 23:58:00   0.027502 -14.043478  22.461097  -2.570029  0.377086
2019-12-31 23:59:00  -0.276184 -13.043467  23.760436  -2.577161 -0.007132

[9413280 rows x 5 columns]


In [14]:
###Step 4: save as a csv file.
PATH = '/mnt/c/users/s1725621/Documents/PhDCode/Git/dBdtCorrelation/data_merged/'

###Save to netcdf file
#MAY NOT BE IN CORRECT FORMAT. May require xarray.
# Convert the DataFrame to an xarray Dataset
allobsdbdtx = xr.Dataset.from_dataframe(allobsdbdt)

# Save the Dataset to a NetCDF file
allobsdbdtx.to_netcdf(path = PATH+'mergeddatafromobsdata.nc')

###Save to csv file
# Save the DataFrame to a CSV file
allobsdbdt.to_csv(PATH+'mergeddatafromobsdata.csv',index=True)

# Correlation of different stations dBdt values.

In [16]:
###Require cell here to import from csv exported above.
#PATH = '/mnt/c/users/s1725621/Documents/PhDCode/Git/dBdtCorrelation/data_merged/'

#allobsdbdt2=pd.read_csv(PATH + 'mergeddatafromobsdata.csv',index_col='Timestamp')###Observatory Dataset downloaded

#my_data = np.genfromtxt(PATH + 'mergeddatafromobsdata.csv', delimiter=',')

###import from netcdf
# Specify the path to the NetCDF file
nc_path = PATH + 'mergeddatafromobsdata.nc'

# Open the NetCDF file
nc_data = xr.open_dataset(nc_path)

# Convert to a Pandas DataFrame
df = nc_data.to_dataframe()

In [17]:
#print(allobsdbdt)
print(df)

                          ABG       ABK           BDV       BEL       BLC  \
index                                                                       
1998-01-01 00:00:00       NaN       NaN           NaN       NaN       NaN   
1998-01-01 00:01:00       NaN  0.308327 -2.972726e-01 -0.512901 -1.178839   
1998-01-01 00:02:00       NaN -0.655964 -2.972721e-01 -0.470311  1.877301   
1998-01-01 00:03:00       NaN  0.720416 -2.999013e-01  0.134179  0.599422   
1998-01-01 00:04:00       NaN  0.723871 -2.636033e-03 -0.347471 -1.086234   
...                       ...       ...           ...       ...       ...   
2019-12-31 23:55:00 -0.001343  0.410346  7.248893e-03 -0.210084 -0.086111   
2019-12-31 23:56:00  0.000709  0.196838 -1.287481e-07  0.011086 -0.085835   
2019-12-31 23:57:00  0.098654 -0.016670  9.281893e-02 -0.099013 -0.094947   
2019-12-31 23:58:00  0.101458  0.131442 -7.248625e-03  0.076883  0.513625   
2019-12-31 23:59:00 -0.001891  0.131685 -7.579840e-03  0.065488  0.490284   

In [44]:
# Drop rows with missing values
allobsdbdt.dropna(inplace=True)

# Perform k-means clustering
k = 5  # Number of clusters
kmeans = KMeans(n_clusters=k)

#transpose the data.
allobsdbdt=allobsdbdt.T

kmeans.fit(allobsdbdt)

/home/spiralwidget/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5)

In [ ]:
# Get cluster centers
cluster_centers = kmeans.cluster_centers_

# Assign cluster labels to each time series
cluster_labels = kmeans.labels_

# Plot the clustered time series
plt.figure(figsize=(10, 5))
for i in range(k):
     plt.plot(allobsdbdt.columns, cluster_centers[i], label=f'Cluster {i+1} Center', marker='o')
plt.title('Clustered Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

# Display cluster labels
print("Cluster Labels:", cluster_labels)

In [ ]:
cluster_centers=np.array(cluster_centers)

kcluster=[]


for i in range(len((cluster_centers[0]))):
    ticker=0
    j=min(cluster_centers[:,i])
    for k in cluster_centers[:,i]:
        if j== k:
            kcluster.append(ticker)
        else:
            ticker+=1

In [ ]:
# Plot the clustered time series
plt.figure(figsize=(10, 5))
plt.plot(allobsdbdt.columns, kcluster, marker='o')
plt.title('Clustered Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

# Display cluster labels
print("Cluster Labels:", cluster_labels)